In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install ktrain
!pip install tf_keras
import pandas as pd
import numpy as np
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [ ]:
import os
os.environ['TF_USE_LEGACY_KERAS'] = '1'
import ktrain
from ktrain import text

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/Political Multiclass Sentiment Analysis of Tamil/PS_train.csv')
val_df =pd.read_csv('/content/drive/MyDrive/Political Multiclass Sentiment Analysis of Tamil/PS_dev.csv')
test_df = pd.read_csv('/content/drive/MyDrive/Political Multiclass Sentiment Analysis of Tamil/PS_test_without_lables.csv')

In [ ]:
from sklearn.utils import resample
import random

none_above_df = train_df[train_df['labels'] == 'None of the above']
none_above_resampled = none_above_df.sample(n=300, replace=True, random_state=42)
train_df = pd.concat([train_df, none_above_resampled], ignore_index=True)

In [ ]:
train_text = train_df['content']
val_text = val_df['content']
test_text = test_df['content']

train_labels = train_df['labels']
val_labels = val_df['labels']
# test_labels = test_df['labels']

In [ ]:
import re

def preprocess_tamil_text(text):
    # Remove hashtags, question marks, commas, and periods
    text = re.sub(r"[#?,.]", "", text)
    text = text.lower()
    # Remove emojis (all unicode symbols in emoji range)
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese characters
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642"
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r"", text)

    # Remove extra spaces
    text = re.sub(r"\s+", " ", text).strip()
    # words = text.split()

    # Filter out stop words
    # text = [word for word in words if word not in tamil_stopwords]
    return text

# Example usage
# tamil_text = "தமிழ் #மொழி மிகவும் சிறந்ததா? 😊, ஆனால் இது படிக்க சுவாரஸ்யமாக உள்ளது."
# preprocessed_text = preprocess_tamil_text(tamil_text)
# print(preprocessed_text)


In [ ]:
# Applying preprocessing to train, val, and test data
train_text = train_df['content'].apply(preprocess_tamil_text)
val_text = val_df['content'].apply(preprocess_tamil_text)
test_text = test_df['content'].apply(preprocess_tamil_text)

In [ ]:
# Display preprocessed text
print("Train Text:\n", train_text.head())
print("Validation Text:\n", val_text.head())
print("Test Text:\n", test_text.head())

Train Text:
 0    தென்காசி தொகுதி புதிய தமிழகம் கட்சி வேட்பாளர் ...
1    அண்ணன் இதனை சூசகமாக 11 மாதங்கள் முன்பே பேட்டிய...
2    ஒரு வருடம் ஆகி விட்டது இந்த துயரம் நேர்ந்து இன...
3    எடப்பாடியை கண்டுகொள்ளாத "எடப்பாடி" --- ஆதரிப்ப...
4    எங்களின் அரசியல் அடுத்த தலைமுறைக்குமானது மக்கள...
Name: content, dtype: object
Validation Text:
 0    ஒற்றே அறை பாஜக குளோஸ்! மக்களின்_சின்னம்_மைக் ச...
1    வாய்ப்பில்லை சார் வாய்ப்பில்லை சீமானின்_சின்னம...
2    ஒரே போடு இரண்டாக பிளக்கும் | வரலாற்றில் ஒருவன்...
3    இப்படியெல்லாம் எடிட் பண்ணிதான் ஓட்டு வாங்கணும்...
4    mike_voiceofpeople ntk_symbol_mike மக்களின்_சி...
Name: content, dtype: object
Test Text:
 0    இஸ்லாமிய சகோதரர்களுடன் ரமலான் கொண்டாடிய அதிமுக...
1    ஓபிஎஸ் - எடப்பாடி போட்டா போட்டி! திடீரென பணிகள...
2    இன்றைய பரப்புரை: நாம் தமிழர் கட்சி தலைமை ஒருங்...
3    இன்னும் 05 ஏ நாளில் வெல்வோம் தமிழராய் - தலை நி...
4    டாஸ்மாக்ல மட்டும்தான் கருணாநிதி அவர்களின் பெயர...
Name: content, dtype: object


In [ ]:
train_labels.value_counts()

,count
labels,
Opinionated,1361
Sarcastic,790
Neutral,637
Positive,575
None of the above,471
Substantiated,412
Negative,406


In [ ]:
# train_labels = train_labels.fillna("Unknown").apply(lambda x: classes_list.index(x) if x in classes_list else -1)
# val_labels = val_labels.fillna("Unknown").apply(lambda x: classes_list.index(x) if x in classes_list else -1)


In [ ]:
classes_list = ["Opinionated", "Sarcastic", "None of the above", "Neutral", "Positive", "Substantiated", "Negative"]
# Map the labels to their corresponding indices
train_labels = train_labels.apply(lambda x: classes_list.index(x))
val_labels = val_labels.apply(lambda x: classes_list.index(x))


In [ ]:
# MODEL_NAME = 'distilbert-base-multilingual-cased'
MODEL_NAME = 'FacebookAI/xlm-roberta-base'
t = text.Transformer(MODEL_NAME, maxlen=30, class_names=classes_list)
trn = t.preprocess_train(np.array(train_text), np.array(train_labels))
val = t.preprocess_test(np.array(val_text), np.array(val_labels))
model = t.get_classifier()
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=32)

preprocessing train...
language: ta
train sequence lengths:
	mean : 17
	95percentile : 31
	99percentile : 40


Is Multi-Label? False
preprocessing test...
language: ta
test sequence lengths:
	mean : 17
	95percentile : 31
	99percentile : 39


In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
filepath = "model"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [ ]:
learner.fit_onecycle(5e-5, 20, verbose=2, callbacks=callbacks_list)



begin training using onecycle policy with max lr of 5e-05...
Epoch 1/20

Epoch 1: val_accuracy improved from -inf to 0.30515, saving model to model
146/146 - 214s - loss: 1.8265 - accuracy: 0.2934 - val_loss: 1.7324 - val_accuracy: 0.3051 - 214s/epoch - 1s/step
Epoch 2/20

Epoch 2: val_accuracy improved from 0.30515 to 0.30699, saving model to model
146/146 - 203s - loss: 1.6783 - accuracy: 0.3637 - val_loss: 1.7264 - val_accuracy: 0.3070 - 203s/epoch - 1s/step
Epoch 3/20

Epoch 3: val_accuracy improved from 0.30699 to 0.31250, saving model to model
146/146 - 180s - loss: 1.6060 - accuracy: 0.3850 - val_loss: 1.6866 - val_accuracy: 0.3125 - 180s/epoch - 1s/step
Epoch 4/20

Epoch 4: val_accuracy improved from 0.31250 to 0.34191, saving model to model
146/146 - 182s - loss: 1.5599 - accuracy: 0.4033 - val_loss: 1.6685 - val_accuracy: 0.3419 - 182s/epoch - 1s/step
Epoch 5/20

Epoch 5: val_accuracy improved from 0.34191 to 0.34559, saving model to model
146/146 - 181s - loss: 1.5372 - ac

In [ ]:
learner.validate(class_names=t.get_classes())

17/17 [==============================] - 4s 109ms/step
                   precision    recall  f1-score   support

      Opinionated       0.29      0.98      0.45       153
        Sarcastic       0.20      0.01      0.02       115
None of the above       0.57      0.85      0.68        20
          Neutral       0.00      0.00      0.00        84
         Positive       0.00      0.00      0.00        69
    Substantiated       0.00      0.00      0.00        52
         Negative       0.00      0.00      0.00        51

         accuracy                           0.31       544
        macro avg       0.15      0.26      0.16       544
     weighted avg       0.15      0.31      0.16       544



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


array([[150,   2,   1,   0,   0,   0,   0],
       [106,   1,   8,   0,   0,   0,   0],
       [  3,   0,  17,   0,   0,   0,   0],
       [ 82,   0,   2,   0,   0,   0,   0],
       [ 67,   1,   1,   0,   0,   0,   0],
       [ 51,   1,   0,   0,   0,   0,   0],
       [ 50,   0,   1,   0,   0,   0,   0]])

In [ ]:
test_text1 = test_df['content']

In [ ]:
model.load_weights(filepath)
predictor = ktrain.get_predictor(learner.model, preproc=t)
predict = predictor.predict(test_text1.values)

In [ ]:
df = pd.DataFrame({
    'Id': test_df['Id'],
    'Labels': predict
})

In [ ]:
df.to_csv('/content/drive/MyDrive/Political Multiclass Sentiment Analysis of Tamil/MNLP_Tamil_run3', index=False)

In [ ]:
# run1->32 f1 score xlm_roberta
# run2->  25     mbert 64 2e-5  stopwords removed
#run3  ->       xlm  32  5e-5 stopwords removed

In [ ]:
# learner.fit_onecycle(2e-4, 12, verbose=2, callbacks=callbacks_list)



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/12

Epoch 1: val_accuracy improved from -inf to 0.28125, saving model to xlm_roberta_multiclass_sentiment_detection
136/136 - 137s - loss: 1.8583 - accuracy: 0.2617 - val_loss: 1.7800 - val_accuracy: 0.2812 - 137s/epoch - 1s/step
Epoch 2/12

Epoch 2: val_accuracy did not improve from 0.28125
136/136 - 44s - loss: 1.7654 - accuracy: 0.3132 - val_loss: 1.7682 - val_accuracy: 0.2812 - 44s/epoch - 325ms/step
Epoch 3/12

Epoch 3: val_accuracy did not improve from 0.28125
136/136 - 42s - loss: 1.7384 - accuracy: 0.3178 - val_loss: 1.7468 - val_accuracy: 0.2757 - 42s/epoch - 312ms/step
Epoch 4/12

Epoch 4: val_accuracy improved from 0.28125 to 0.33640, saving model to xlm_roberta_multiclass_sentiment_detection
136/136 - 90s - loss: 1.6897 - accuracy: 0.3509 - val_loss: 1.6649 - val_accuracy: 0.3364 - 90s/epoch - 663ms/step
Epoch 5/12

Epoch 5: val_accuracy improved from 0.33640 to 0.37132, saving model to xlm_roberta_multi

In [ ]:
# learner.fit_onecycle(2e-5, 10, verbose=2, callbacks=callbacks_list)  #distilbert_mbert



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/10

Epoch 1: val_accuracy improved from -inf to 0.28125, saving model to xlm_roberta_multiclass_sentiment_detection
68/68 - 61s - loss: 1.8319 - accuracy: 0.2888 - val_loss: 1.7799 - val_accuracy: 0.2812 - 61s/epoch - 893ms/step
Epoch 2/10

Epoch 2: val_accuracy improved from 0.28125 to 0.28493, saving model to xlm_roberta_multiclass_sentiment_detection
68/68 - 48s - loss: 1.7518 - accuracy: 0.3159 - val_loss: 1.7596 - val_accuracy: 0.2849 - 48s/epoch - 710ms/step
Epoch 3/10

Epoch 3: val_accuracy improved from 0.28493 to 0.29779, saving model to xlm_roberta_multiclass_sentiment_detection
68/68 - 40s - loss: 1.7382 - accuracy: 0.3297 - val_loss: 1.7367 - val_accuracy: 0.2978 - 40s/epoch - 589ms/step
Epoch 4/10

Epoch 4: val_accuracy improved from 0.29779 to 0.29963, saving model to xlm_roberta_multiclass_sentiment_detection
68/68 - 47s - loss: 1.6969 - accuracy: 0.3412 - val_loss: 1.6999 - val_accuracy: 0.2996 - 47s